# poi별 소셜 버즈량데이터에 날씨 데이터 합치기

[목표]
- 기존의 소셜 버즈 데이터에, 제주시, 서귀포시의 기상 데이터를 추출 후 지역별로 데이터를 합친 뒤, poi별로 csv파일에 저장


[데이터셋]
- final : 이전의 네이버 크롤링 과정을 통해 추출한 모든 관광지의 소셜 버즈 데이터

- jeju_weather : 제주시 2021년 날씨 예측 데이터 (기준 : 아라동)
- seogwipo_weather : 서귀포시 2021년 날씨 예측 데이터 (기준 : 천지동)

[변수 설명]
- final : 기존에 작성해 둔 poi별 소셜버즈량 결합 데이터
- jeju_weather : 2021년 제주시 날씨 예측 데이터
- seogwipo_weather : 2021년 서귀포시 날씨 예측 데이터
- search : 특정 관광지 지정
- a : 관광지명(visit)이 search랑 일치하는 데이터만 가져온 데이터프레임

[날씨 데이터 칼럼 설명]
- Date : 2021-01-01 ~ 2021-12-31 날씨 데이터
- day : 요일을 일정 형태로 지정한 값 (월,화,수,목요일은 0, 금요일은 1, 주말은 2로 표현)
- holiday : 공휴일, 대체공휴일에 해당 날짜가 포함되는지 판단 -> 1이면 포함, 0이면 공휴일이 아님
- minimum temperature(°C) : 최저 기온
- maximum temperature(°C) : 최고기온
- humidity : 습도
- precip_prob(%) : 강수 확률
- precip_type : 강수 유형 (0 -> 없음, 1 -> 비, 2 -> 눈, 3 -> 눈비)

[진행 과정]
- 소셜 버즈 데이터와 날씨 데이터를 결합
    - 단, 해당 관광지의 위치가 제주시이면 제주시의 날씨 데이터와 합치고, 서귀포시이면 서귀포시와 합침
- 합친 데이터를 각 poi별로 분할해 날씨반영poi데이터폴더에 각 poi별로 저장

In [1]:
# 라이브러리
import pandas as pd
import numpy as np

In [2]:
# 미리 소셜 버즈량 합쳐놓은 데이터
final = pd.read_csv('../Dataset/raw_data/social buzz/social buzz final.csv')

# 제주시, 서귀포시 2021년 날씨 예측 데이터
jeju_weather = pd.read_csv('../Dataset/final_data/jeju weather final.csv') # 제주시
seogwipo_weather = pd.read_csv('../Dataset/final_data/seogwipo weather final.csv') # 서귀포시

In [3]:
# 소셜 버즈 데이터 칼럼명 재지정
final.columns = ['visit','date','buzz','place']

In [4]:
# 소셜 버즈 데이터 결과 확인
final.head()

,visit,date,buzz,place
0,1100고지습지,2021-01-01,1,서귀포시
1,1100고지습지,2021-01-02,16,서귀포시
2,1100고지습지,2021-01-03,24,서귀포시
3,1100고지습지,2021-01-04,7,서귀포시
4,1100고지습지,2021-01-05,0,서귀포시


In [5]:
# 제주도 날씨 데이터
jeju_weather.head()

,Unnamed: 0,date,day,holiday,minimum temperature(°C),maximum temperature(°C),humidity(%),precip_prob(%),precip_type
0,0,2021-01-01,1,1,-1,5,75,30,0
1,1,2021-01-02,2,0,0,4,75,30,0
2,2,2021-01-03,2,0,3,7,75,30,0
3,3,2021-01-04,0,0,-1,5,75,0,0
4,4,2021-01-05,0,0,-4,2,75,30,0


In [6]:
# 서귀포시 날씨 데이터
seogwipo_weather.head()

,Unnamed: 0,date,day,holiday,minimum temperature(°C),maximum temperature(°C),humidity(%),precip_prob(%),precip_type
0,0,2021-01-01,1,1,3,11,60,30,0
1,1,2021-01-02,2,0,5,10,70,20,0
2,2,2021-01-03,2,0,7,12,70,20,0
3,3,2021-01-04,0,0,3,12,60,0,0
4,4,2021-01-05,0,0,1,8,65,20,0


In [7]:
# 불필요 칼럼 제거
jeju_weather.drop('Unnamed: 0',axis=1,inplace=True)
seogwipo_weather.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
# 소셜 버즈 데이터 총 관광지 개수
final['visit'].nunique()

121

In [9]:
# 첫번째 관광지를 search로 지정
search = final['visit'].unique()[0]

In [10]:
# 관광지명이 search로 지정된 관광지만 가져옴
a = final[final['visit']==search]

In [11]:
# 해당 관광지의 시 파악
a['place'][0]

'서귀포시'

In [12]:
# place칼럼 제거
a.drop('place',axis=1,inplace=True)

C:\Users\LeeSH\AppData\Local\Temp\ipykernel_21208\2292273159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a.drop('place',axis=1,inplace=True)


In [13]:
# place 칼럼 제거한 결과 확인
a

,visit,date,buzz
0,1100고지습지,2021-01-01,1
1,1100고지습지,2021-01-02,16
2,1100고지습지,2021-01-03,24
3,1100고지습지,2021-01-04,7
4,1100고지습지,2021-01-05,0
...,...,...,...
360,1100고지습지,2021-12-27,7
361,1100고지습지,2021-12-28,7
362,1100고지습지,2021-12-29,3
363,1100고지습지,2021-12-30,5


In [14]:
# 각 관광지별로 제주시이면 제주시의 날씨 데이터를, 서귀포시면 서귀포시의 날씨 데이터를 합쳐서 저장
for i in range(final['visit'].nunique()): # 총 수행 횟수 : final 데이터의 visit칼럼의 중복되지 않는 개수(poi개수)
    search = final['visit'].unique()[i] # poi(관광지) 지정
    df = final[final['visit']==search] # 방문지가 search인 데이터만 추출
    a = df['place'].unique()[0] # 제주시 vs 서귀포시인지 판단 
    if a=='제주시': # a가 제주시인 경우
        # 제주시 날씨 데이터와 결합
        df = pd.merge(df, jeju_weather, how = 'outer', on='date')
        # place칼럼 제거
        df = df.drop('place',axis=1)
        # 파일 저장 (place 칼럼을 지웠기에 명확한 구분을 위해 제주시 관광지의 경우 제주시- 추가)
        df.to_csv('../Dataset/raw_data/social buzz/제주시-{:s}.csv'.format(search), index=False, encoding='utf-8-sig')
    else: # a가 서귀포시인 경우
        # 서귀포시 날씨 데이터와 결합 
        df = pd.merge(df, seogwipo_weather, how = 'outer', on='date')
        # place칼럼 제거
        df = df.drop('place',axis=1)
        # 파일 저장 (place 칼럼을 지웠기에 명확한 구분을 위해 서귀포시 관광지의 경우 서귀포시- 추가)
        df.to_csv('../Dataset/raw_data/social buzz/서귀포시-{:s}.csv'.format(search), index=False, encoding='utf-8-sig')